# Présentation heuristique

# Heuristiques
## CDS
## GUPTA
## NEH

# Colonie de fourmis
### Parametres
1. initValue = $10^{-6}$: valeur initial du pheromone
1. nbAnts = 5: nombre de fourmis
1. rho = 0.01: taux d’évaporation de phéromone
1. alpha = 1: importance pheromone
1. beta = 0.0001, : importance info heuristique 
1. q0 = 0.97:  
    * j = argmax[$phero^{\alpha} * heuristique^{\beta}$]
    * $p_{i,j}= \frac{phero^{\alpha} * heuristique^{\beta}}{\sum{phero^{\alpha} * heuristique^{\beta}}}$  pour j $\in$ $N_i^k$
1. nb_rounds : nombre d'episode
1. parallel : execution parallele
1. threads : Nombre de threads
1. local_search = True, : recherche local
1. local_search_proba = 0.02, : proba de la recherche local pour chaque job
1. local_search_nb_permutation = 3 : nombre de permutation pour chaque job
1. heuristic_info_strategy

# Colonie de fourmis
### Info heuritstique
- Rule 1: (Extension of Johnson's Rule):
Si un job a des temps de traitement plus courts sur les premieres
machines que celles des machines finales, il doit être
traité en premier.
- Rule 2: (Extension of SPT):
si un job a un temp reduit sur toute les machines, il doit etre en premier

### MAJ du pheromone
$\tau_{i,j} = (1-\rho) \tau_{i,j}+\rho* \frac{z}{C_{max}^{best}}$ pour (i,j) $\in$ Best

# Algorithme génétique
### Parametres
1. popul_size : taille de la population
1. nb_generations : nombre de générations
1. Pc = 0.9 : probabilité de crossover
1. Pm = 0.06 : probabilité de mutation

# Algorithme génétique
### Sélection
### Crossover
### Mutation
### MAJ population

# Recuit simulé
### Parametres
1. initial_value = 'CDS': l'heuristique d'initialisation
1. Ti=200 : Température initiale
1. Tf = 0.1 : Température finale
1. nb_repetitions=10 :Nombres d'iteration avant changement de temperature 
1. alpha = 0.93 : Taux de mise a jour

# Tests 

In [2]:
machines = 20
jobs = 50
indtance_id = 0

In [3]:
import json

RESULT_PATH = "../results/"
res = {}
file_name = 'b_'+str(jobs)+'_'+str(machines)+'/res_'+str(indtance_id)+'.json'

methodes_list = ['cds', 'neh', 'aco','sa', 'ga']

for meth in methodes_list:
    with open(RESULT_PATH+meth+'/'+file_name) as f:
            res[meth] = {
                
            } json.load(f)

In [4]:
infos = {}
def fill_info(method_name):
    infos[method_name] = {
        'max' : max(res[method_name], key =lambda x: x['results'][0]['results']['C_max']),
        'min' : min(res[method_name], key =lambda x: x['results'][0]['results']['C_max'])
    } 
    
for meth in methodes_list:
    fill_info(meth)

In [5]:
import pandas as pd
data = []
for k, v in infos.items():
    data.append( [v['max']['results'][0]['results']['C_max'],
              v['min']['results'][0]['results']['C_max']
              ])
    
pd.DataFrame(data, methodes_list, columns=["max", "min"])

,max,min
cds,4319.0,4319.0
neh,5035.0,5035.0
aco,4397.0,4130.0
sa,4511.0,4035.0
ga,4538.0,4048.0


In [6]:
import numpy as np
import matplotlib.pyplot as plt
probas = res['ga'][:40]
result = []
re = []
for i in range(len(probas)):
    val = probas[i]
    if i % 5 == 0 and i !=0:
        result.append(re)
        re = []
    re.append(val['results'][0]['results']['C_max'])
result.append(re)

df = pd.DataFrame(result, [0.8, 0.85, 0.87, 0.9, 0.92, 0.94, 0.96, 0.98], [0.01, 0.02, 0.06, 0.1, 0.2])
df

,0.01,0.02,0.06,0.10,0.20
0.80,4119.0,4267.0,4202.0,4241.0,4228.0
0.85,4200.0,4184.0,4220.0,4180.0,4200.0
0.87,4129.0,4239.0,4166.0,4156.0,4240.0
0.90,4187.0,4197.0,4170.0,4271.0,4308.0
0.92,4245.0,4192.0,4150.0,4246.0,4154.0
0.94,4155.0,4164.0,4130.0,4267.0,4204.0
0.96,4140.0,4192.0,4166.0,4278.0,4150.0
0.98,4156.0,4241.0,4134.0,4168.0,4254.0


# AG

In [1]:
import json
import pandas as pd

RESULT_PATH = "../results/"
res = {}

tai_benchmarks = [
        [20,5],
        [20,10],
        [20,20],
        [50,5],
        [50,10],
        [50,20],
        [100,5],
        [100,10],
#          [100,20],
#         [500,20]
    ]

file_names = []
for ben in tai_benchmarks:
    for inst in range(1):
        file_names.append('b_'+str(ben[0])+'_'+str(ben[1])+'/res_'+str(inst)+'.json')

methodes_list = ['cds', 
                 'neh', 
                 'aco',
#                  'sa', 
                 'ga']

for meth in methodes_list:
    res[meth] = {}
    for file_name in file_names:
        with open(RESULT_PATH+meth+'/'+file_name) as f:
                res[meth][file_name] = json.load(f)

In [2]:
import os, sys
sys.path.append("../")

from utils import Instance, Benchmark,JsonBenchmark, gen_instance_id

def instance_file_to_numbers(file):
    file_split = file.split('_')
    return {
        'jobs_number': int(file_split[1]),
        'machines_number': int(file_split[2]),
        'instance_number': int(file_split[3][0])
    } 

def benchmark_file_to_numbers(file):
    #file exemple tai5_5.txt we want to extract (5, 5)
    file = file[3:-4]
    file_split = file.split('_')
    return {
        'jobs_number': int(file_split[0]),
        'machines_number': int(file_split[1]),
    } 

def read_all_instances(BENCHMARKS_FOLDER):
    results = {}
    instance_id = ''
    for file in os.listdir(BENCHMARKS_FOLDER):
        if file.endswith(".txt"):
            result = benchmark_file_to_numbers(file)
            benchmark = Benchmark(result['jobs_number'], result['machines_number'], BENCHMARKS_FOLDER)
            instances_number = benchmark.get_instances_number()
            for i in range(instances_number):
                inst = benchmark.get_instance(i)
                result['jobs_number'], result['machines_number'], i
                inst.id = 'b_'+str(result['jobs_number'])+'_'+str(result['machines_number'])+'/res_'+str(i)+'.json'
                results[inst.id] = inst
    return results

#key:id instance -> value: Instance object
instances = read_all_instances('../benchmarks')

In [3]:
ga = [0 for _ in range(len(res['ga']['b_20_20/res_0.json']))]
for instance, result in res['ga'].items():
    # 40 are the results where we used defferent pc and pm
    for i in range(len(result)):
        one_instance = result[i]
        ga[i] += one_instance['results'][0]['results']['C_max'] / instances[instance].lower_bound

## Pc vs Pm

In [4]:
import numpy as np
import matplotlib.pyplot as plt

result = []
re = []
for i in range(len(ga[:40])):
    val = ga[i]
    if i % 5 == 0 and i !=0:
        result.append(re)
        re = []
    re.append(val)
result.append(re)
df = pd.DataFrame(result, [0.8, 0.85, 0.87, 0.9, 0.92, 0.94, 0.96, 0.98], [0.01, 0.02, 0.06, 0.1, 0.2])
df

,0.01,0.02,0.06,0.10,0.20
0.80,8.804868,8.955491,8.841889,8.908552,8.887454
0.85,8.996020,8.929938,8.928470,8.836845,8.902091
0.87,8.900080,8.912847,8.886084,8.829580,8.909423
0.90,8.877612,8.874201,8.828810,8.861719,8.887377
0.92,8.888275,8.898121,8.874821,8.923960,8.853282
0.94,8.924514,8.889257,8.897260,8.915835,8.888774
0.96,8.840277,8.894800,8.872787,8.857395,8.809398
0.98,8.846332,8.876691,8.856568,8.839123,8.921467


## popul_size vs nb_generations

In [5]:
import numpy as np
import matplotlib.pyplot as plt

result = []
re = []
for i in range(len(ga[40:])):
    val = ga[i]
    if i % 9 == 0 and i !=0:
        result.append(re)
        re = []
    re.append(val)
result.append(re)

df = pd.DataFrame(result, [5, 7, 9, 12, 15, 20, 50, 80], [20, 30, 50, 70, 100, 120, 150, 170, 200])
df

,20,30,50,70,100,120,150,170,200
5,8.804868,8.955491,8.841889,8.908552,8.887454,8.996020,8.929938,8.928470,8.836845
7,8.902091,8.900080,8.912847,8.886084,8.829580,8.909423,8.877612,8.874201,8.828810
9,8.861719,8.887377,8.888275,8.898121,8.874821,8.923960,8.853282,8.924514,8.889257
12,8.897260,8.915835,8.888774,8.840277,8.894800,8.872787,8.857395,8.809398,8.846332
15,8.876691,8.856568,8.839123,8.921467,9.246678,9.317560,9.141697,9.125426,9.064394
20,9.032215,9.013897,8.980508,8.941498,9.327715,9.298575,9.176522,9.121591,9.135596
50,8.959271,8.952230,8.898556,8.916960,9.283979,9.245798,9.140231,9.020949,9.003453
80,8.942837,8.896200,8.915258,8.894924,9.239390,9.129544,9.014998,8.953136,8.932534
